In [3]:
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

from astropy import units as u
from astropy.coordinates import SkyCoord

#http://simbad.u-strasbg.fr/simbad/sim-fsam
from astroquery.simbad import Simbad
from astropy.coordinates import EarthLocation
from astropy.coordinates import AltAz

from astroplan import Observer
from astroplan import download_IERS_A
download_IERS_A()

import glob
import pandas as pd

from ipywidgets import interact, fixed
import ipywidgets as widgets

from current_utils_svn import *
import os

In [20]:
def plot_current_image(current_reading_list_mod, ridx, offset_x, offset_y, save_offsets):

    fig = plt.figure(figsize=(20,10))
    ax = plt.subplot(1,2,1)
    ax.set_xlim(-25,25)
    ax.set_ylim(-25,25)
    for k in range(25):
        xx, yy = np.meshgrid(x_list[k],y_list[k])
        xx_pos, yy_pos = np.meshgrid(x_list_pos[k],y_list_pos[k])
        c = ax.pcolor(xx,yy,current_reading_list_mod[ridx][yy_pos,xx_pos],vmin=0,vmax=800, cmap='viridis')
    x_B = y_B = np.asarray([i - 19.5 + 0.31 * ((i // 8) - 2) for i in range(40)]).T
    x_B_mg, y_B_mg = np.meshgrid(x_B,y_B)
    ax.plot(x_B_mg, y_B_mg, 'o',color='white', markersize=2)
    ax.plot(0.,0.,'r+',markersize=1000)
    
    if(np.str(camera_coord_pix_cameraview_list[ridx])!='nan'):
        for j in range(len(camera_coord_pix_cameraview_list[ridx])):
            ax.plot(camera_coord_pix_cameraview_list[ridx][j][0],
                   camera_coord_pix_cameraview_list[ridx][j][1],'r*',label=star_names_list[ridx][j],markersize=10)
            ax.text(camera_coord_pix_cameraview_list[ridx][j][0],
                   camera_coord_pix_cameraview_list[ridx][j][1],star_names_list[ridx][j],size=15,color='orange',
                    horizontalalignment='center',verticalalignment='top')
        
    ax.set_title('{}, reading: {}, time: {}'.format(run_list[run_index], ridx, df_timestamp_currents[ridx])
                 , fontsize=15)
    
    fig.colorbar(c, ax=ax)

    ax = plt.subplot(1,2,2)
    ax.set_xlim(-25,25)
    ax.set_ylim(-25,25)
    for k in range(25):
        xx, yy = np.meshgrid(x_list[k],y_list[k])
        xx_pos, yy_pos = np.meshgrid(x_list_pos[k],y_list_pos[k])
        c = ax.pcolor(xx,yy,current_reading_list_mod[ridx][yy_pos,xx_pos],vmin=0,vmax=800, cmap='viridis')
    x_B = y_B = np.asarray([i - 19.5 + 0.31 * ((i // 8) - 2) for i in range(40)]).T
    x_B_mg, y_B_mg = np.meshgrid(x_B,y_B)
    ax.plot(x_B_mg, y_B_mg, 'o',color='white', markersize=2)
    ax.plot(0.,0.,'r+',markersize=1000)
    ax.plot(offset_x,offset_y,'b+',markersize=1000)
    
    if(np.str(camera_coord_pix_cameraview_list[ridx])!='nan'):
        for j in range(len(camera_coord_pix_cameraview_list[ridx])):
            ax.plot(camera_coord_pix_cameraview_list[ridx][j][0]+offset_x,
                   camera_coord_pix_cameraview_list[ridx][j][1]+offset_y,'r*',label=star_names_list[ridx][j],markersize=10)
            ax.text(camera_coord_pix_cameraview_list[ridx][j][0]+offset_x,
                   camera_coord_pix_cameraview_list[ridx][j][1]+offset_y,star_names_list[ridx][j],size=15,color='orange',
                    horizontalalignment='center',verticalalignment='top')
        
    ax.set_title('{}, reading: {}, time: {}'.format(run_list[run_index], ridx, df_timestamp_currents[ridx])
                 , fontsize=15)
    
    fig.colorbar(c, ax=ax)
    
    if (save_offsets==True):
        a_offsets_x[ridx] = offset_x
        print("saving offset_x={0:.2f} to reading {1:d}".format(offset_x,ridx))
        print("a_offsets_x",a_offsets_x)
        a_offsets_y[ridx] = offset_y
        print("saving offset_y={0:.2f} to reading {1:d}".format(offset_y,ridx))
        print("a_offsets_y",a_offsets_y)
        try:
            os.makedirs(wdir+plotdir+"run{0}/".format(run_list[run_index]))
        except FileExistsError:
            # directory already exists
            pass
        plt.savefig(wdir+plotdir+"run{0}/{1}_reading={2}_meshgrid_widget.pdf".format(run_list[run_index],
                                                                                   run_list[run_index],
                                                                                   ridx))

In [18]:
wdir = "/Users/massimocapasso/Documents/BarnardCollege/CTA/SCT/Camera/"
datadir = "pSCTdata/"
logs_dir = "positioner_logs/"
plotdir = "plots/"

run_list = pd.read_excel(wdir+"poslog_currlog.xlsx")['run']
fname_list = pd.read_excel(wdir+"poslog_currlog.xlsx")['poslog']
fname_currents_list = ["{}_currents.txt".format(run) for run in run_list]

In [6]:
run_index = 21

print(run_list[run_index])

fname = fname_list[run_index]
fname_currents = fname_currents_list[run_index]

print(fname)
print(fname_currents)

"""
log df
"""
df = pd.read_csv(wdir+datadir+logs_dir+fname,parse_dates=['current_time','current_Time_DT'],
                 infer_datetime_format=True)

df_timestamp_key = 'current_Time_DT'
df = df.set_index(df_timestamp_key) #allows search with DataFrame.between_time if needed
df_timestamp = df.index #now the df_timestamp_key is the index

"""
currents df
"""
title_currents = fname_currents.split('.')[0].split('_')[0]

df_currents = pd.read_csv(wdir+datadir+fname_currents,parse_dates=['timestamp_start','timestamp_end'],
                          infer_datetime_format=True)
shape = df_currents.shape
df_currents_keys = df_currents.keys()
#check for pixels with readings below 0 and replace them with nan
for key in df_currents_keys[2:]:
    if(key.split('_')[1]=='pixelglobal'):
        continue
    #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
    for i in range(len(df_currents.loc[:,key])):
        if(df_currents.loc[i,key] < 0):
            df_currents.loc[i,key] = np.nan

df_timestamp_currents_key = 'timestamp_start'
df_currents = df_currents.set_index(df_timestamp_currents_key) #allows search between times
df_timestamp_currents = df_currents.index #now the df_timestamp_key is the index

328629
positionerLog_20200127_194509.txt
328629_currents.txt


# pixels

In [7]:
x_list = []
y_list = []
x_list_pos = []
y_list_pos = []
for m in range(5):
    for k in range(5):
        x_list.append(np.asarray([i - 20 + 0.31 * ((i // 9) - 2) -1*(i//9) for i in range(9*k,9*(k+1))]).T)
        y_list.append(np.asarray([j - 20 + 0.31 * ((j // 9) - 2) -1*(j//9) for j in range(9*m,9*(m+1))]).T)
        x_list_pos.append(np.asarray([i for i in range(8*k,8*(k+1))]).T)
        y_list_pos.append(np.asarray([j for j in range(8*m,8*(m+1))]).T)

In [8]:
ts_list = []
current_position_az_list = []
current_position_alt_list = []

df_dt = (df_timestamp[1] - df_timestamp[0]).total_seconds() #calculates timedelta between pointing readings 
                                                            #assuming it does not change throughout the run
    
#print(df_dt)
for index_reading in range(len(df_timestamp_currents)): 
    #print(index_reading)
    #pick the positioner reading which is closest to each current reading, in an interval timestamp_current+-delta
    #where delta is the time difference between pointing readings
    t0 = df_timestamp_currents[index_reading] - pd.Timedelta(df_dt,unit='second')
    t1 = df_timestamp_currents[index_reading] + pd.Timedelta(df_dt,unit='second')

    t_mask = (df_timestamp > t0) & (df_timestamp < t1)
    
    if(len(np.where(t_mask==True)[0])!=0):
        #accounts for missing pointing readings: e.g.: run328564, index_reading == 0

        index_t = np.argmin(np.abs((df.loc[t_mask].index - df_timestamp_currents[index_reading]).total_seconds()))

        #for the closest positioner reading, extract az,el as well as timestamp
        current_position_az = df.loc[t_mask].iloc[index_t].loc['current_position_az']
        current_position_alt = df.loc[t_mask].iloc[index_t].loc['current_position_el']

        current_position_az_list.append(current_position_az)
        current_position_alt_list.append(current_position_alt)

        
        ts = df.loc[t_mask].index[index_t]
        ts_list.append(ts)
    else:
        current_position_az_list.append(np.nan)
        current_position_alt_list.append(np.nan)
        ts_list.append(np.nan)

In [9]:
"""
create current images for each current readout - exclude modules 4,1,103,125,126,9
"""
modules = list_modules(df_currents)
current_reading_list_mod = []
for reading in range(shape[0]):
    currents = get_currents(df_currents,modules,reading)

    #add central module
    currents_110 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[110] = currents_110
    #exclude modules 4,1,103,125,126,9
    currents_4 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[4] = currents_4
    currents_1 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[1] = currents_1
    currents_103 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[103] = currents_103
    currents_125 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[125] = currents_125
    currents_126 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[126] = currents_126
    currents_9 = currents_by_position = arrange_pixels(np.full(64, np.nan))
    currents[9] = currents_9

    #currents
    currents_0 = np.concatenate([currents[4],currents[5],currents[1],currents[3],currents[2]],axis=1)
    currents_1 = np.concatenate([currents[103],currents[125],currents[126],currents[106],currents[9]],axis=1)
    currents_2 = np.concatenate([currents[119],currents[108],currents[110],currents[121],currents[8]],axis=1)
    currents_3 = np.concatenate([currents[115],currents[123],currents[124],currents[112],currents[7]],axis=1)
    currents_4 = np.concatenate([currents[100],currents[111],currents[114],currents[107],currents[6]],axis=1)
    
    currents_all = np.concatenate([currents_4,currents_3,currents_2,currents_1,currents_0],axis = 0)
    
    current_reading_list_mod.append(currents_all)

In [10]:
observing_location = EarthLocation(lat='31d40m30s', lon='-110d57m7s', height=1268*u.m) #https://en.wikipedia.org/wiki/VERITAS
aa_list = [AltAz(location=observing_location, obstime=observing_time) if(np.str(observing_time)!='nan') else np.nan for observing_time in ts_list]

In [12]:
#define width and height of the box where we want to search for stars
width = 240*u.arcmin
height = 240*u.arcmin

star_names_list = []
camera_coord_pix_cameraview_list = []

for ridx in range(shape[0]):
    if(np.str(ts_list[ridx])=='nan'):
        star_names_list.append(np.nan)
        camera_coord_pix_cameraview_list.append(np.nan)
        continue
    #take center from alt,az pointer log, at the time closest to corresponding current reading
    target_center = SkyCoord(az=current_position_az_list[ridx]*u.deg,alt=current_position_alt_list[ridx]*u.deg,
                      frame='altaz',location=observing_location,obstime=ts_list[ridx])
    target_center_string = target_center.icrs.to_string('hmsdms')

    #query Simbad by criteria: stars(*) with defined magnitude range, within the box
    otype = '*'
    vmag = 7
    search_string = 'region(box,{0},{1}m {2}m) & otype={3} & Vmag <= {4}'.format(
        target_center_string,width.value,height.value,otype,vmag)
    #print(search_string)
    simbad_table = Simbad.query_criteria(search_string)

    
    coo_simbad_list = []
    for i in range(len(simbad_table)):
        coo_simbad_list.append(SkyCoord(simbad_table['RA'][i]+" "+simbad_table['DEC'][i],unit=(u.hourangle, u.deg), frame='icrs'))
    star_names_list.append([star_name.decode("utf-8") for star_name in simbad_table['MAIN_ID']]) #assuming 
                                                                                                  #utf-8 encoding 
                                                                                                  #for simbad table
    """
    find alt az for each of the pointings for each of the stars centroids in the list
    """
    aa = aa_list[ridx]
    center_list = []
    for center in (coo_simbad_list):
        center_list.append(center.transform_to(aa))

    """
    knowing the telescope pointing alt and az, calculate star camera
    coordinates in degrees
    """

    camera_coord_deg_skyview_list = []
    a_tel_el_az = np.array([target_center.alt.radian, target_center.az.radian])
    for center_aa in center_list:
        a_el_az = np.array([center_aa.alt.radian, center_aa.az.radian])
        a_camera_coord = ConvertElevAzimToCameraCoord(a_el_az,a_tel_el_az)
        a_camera_coord_deg = a_camera_coord*180/np.pi
        camera_coord_deg_skyview_list.append(a_camera_coord_deg)

    #print(camera_coord_deg_skyview_list)

    #convert to cameraview
    #camera view is flipped 180 deg around y: x-->-x
    camera_coord_deg_cameraview_list = []
    for a_camera_coord_deg_skyview in camera_coord_deg_skyview_list:
        a_camera_coord_deg_cameraview = np.array([-1*a_camera_coord_deg_skyview[0],a_camera_coord_deg_skyview[1]])
        camera_coord_deg_cameraview_list.append(a_camera_coord_deg_cameraview)

    #print(camera_coord_deg_cameraview_list)

    #convert to pixel coordinates
    camera_coord_pix_cameraview = []
    for a_camera_coord_deg_cameraview in camera_coord_deg_cameraview_list:
        a_camera_coord_pix_cameraview = (a_camera_coord_deg_cameraview)/0.067
        camera_coord_pix_cameraview.append(a_camera_coord_pix_cameraview)

    camera_coord_pix_cameraview_list.append(camera_coord_pix_cameraview)

In [13]:
a_offsets_x = np.empty(len(current_reading_list_mod))
a_offsets_x[:] = np.nan
a_offsets_y = np.empty(len(current_reading_list_mod))
a_offsets_y[:] = np.nan

In [14]:
slider_offset_x = widgets.FloatSlider(value = 0., min=-20., max=20., step = 0.1)
slider_offset_y = widgets.FloatSlider(value = 0., min=-20., max=20., step = 0.1)#, orientation='vertical')
slider_ridx = widgets.IntSlider(value = 0., min=0., max=len(current_reading_list_mod)-1, step=1)
toggle_save_offsets = widgets.ToggleButton(value=False, 
                                           description='Save Offsets', 
                                           disabled=False,
                                           button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                           tooltip='Description',
                                           icon='check')

In [21]:
interact(plot_current_image, current_reading_list_mod=fixed(current_reading_list_mod), 
         ridx=slider_ridx, offset_x=slider_offset_x, offset_y=slider_offset_y,
        save_offsets=toggle_save_offsets);

A Jupyter Widget

In [39]:
a_offsets_x

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan])

In [40]:
a_offsets_y

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan])

In [1201]:
save_length = len((np.where(np.isnan(a_offsets_x)==False))[0])
save_indices = (np.where(np.isnan(a_offsets_x)==False))[0]

In [22]:
#offset: distance from center of star field to center of FOV in pixel coords, skyview
df_offsets = pd.DataFrame({'run':np.array([run_list[run_index]]*save_length),
                           'current_reading_index':save_indices,
                           'timestamp':df_timestamp_currents[save_indices],
                           'delta_x_pix':a_offsets_x[save_indices],
                           'delta_y_pix':-a_offsets_y[save_indices],
                           'timestamp_pointing':df_timestamp[save_indices],
                           'current_pos_az':np.array(current_position_az_list)[save_indices],
                           'current_pos_el':np.array(current_position_alt_list)[save_indices]
                           })

NameError: name 'save_length' is not defined

In [1203]:
if(os.path.exists(wdir+"pointing_corrections_widget.csv")==False):
    df_offsets.to_csv(wdir+"pointing_corrections_widget.csv",sep=',',index=False)
else:
    with open(wdir+"pointing_corrections_widget.csv",'a') as f:
        f.write('\n')
    df_offsets.to_csv(wdir+"pointing_corrections_widget.csv",sep=',',mode='a',index=False,header=False)